In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
%run -i ../../core.py

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.var(dummy_data_tensor, correction = 0)
dummy_data_mean = torch.mean(dummy_data_tensor)

In [5]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = dummy_data_mean, requires_grad = False)

    def forward(self,X):
        # some expression of tolerance to error in the inference
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<0.01*X.size()[1]*(self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*X.size()[1])<0.01*self.w*X.size()[1],x_mean_cons ),self.w)

verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [6]:
# Prover/ data owner side
theory_output = torch.var(data_tensor, correction = 0)
data_mean = torch.mean(data_tensor)
print("theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = data_mean, requires_grad = False)
    def forward(self,X):
        # some expression of tolerance to error in the inference
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<0.01*X.size()[1]*(self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*X.size()[1])<0.01*self.w*X.size()[1],x_mean_cons ),self.w)

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, "default", "resources", settings_path)

theory output:  tensor(211.8691)
==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[0,20332],"logrows":15,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":1512,"total_const_size":0,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [7]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0


==== setting up ezkl ====


spawning module 2
spawning module 0
spawning module 2


Time setup: 2.55598783493042 seconds
Theory output:  tensor(211.8691)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 211.75
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12907834141446617622, 16863260785082668294, 2215826187815521673, 2191547160659437828], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [988086122984763008, 8331390676852675036, 9456440076381036992, 2639995154681636326]]], 'proof': '10b565888946b3178c74ecc886d63dde7734c329dc55fc21f17a8bed2b972a9f13b9837c66faa09c14f2faba61d9ab70adc9faa0fb721ca9d69f517ccf46bf2f1e5ec51a94df5c6b25be69b6c404cd7c96c2422c65931c8c3a0bbefdf5035c660d36061471344f79a5fb61af65e1e38c1589e852b4ac22b86c22bbfa457ed3d6004d95f4fa5d2c26cffbcbeb8ad9cca429a2f1c68fc9896cc199a8385954b7700adbff116e61b3a14828f0fcad09c75ca8d3b3916991dfa3beae2db8d1a3e0bc1fe7bcfd23dd43574a4906d757a65fbfa1df69b482fa78b19e12f82c26d777961444b59c5b8de7eaa27fc85706c88de8f3591b384a979edf30583f318a8671d1145ae1548d5faf16f7ca4349cc830f9a4527da914e07943213853efbb1a5b7af0b9b53e13d4416bfebfc52271ab21aaa30d72e1c08dfb1b836426ffa182eeb00302030e54f1ee03f40ff3d291f0796c1361edc91fe70593e5c128f6fcdd07138

In [8]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[12907834141446617622, 16863260785082668294, 2215826187815521673, 2191547160659437828], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [988086122984763008, 8331390676852675036, 9456440076381036992, 2639995154681636326]]]
proof boolean:  1.0
proof result 1 : 211.75
verified
